<a href="https://colab.research.google.com/github/christotsvetanov/bggpt-google-colab/blob/main/BgGPT_in_google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparation
To speed up the loading of the model it is stored in Google Drive.
The free tier allows using of 15Gb memory - which is enough to holds the entire model.

1. Load the model in your local computer by executing the command:
---
`git clone https://huggingface.co/INSAIT-Institute/BgGPT-7B-Instruct-v0.1`

---
2. Delete folder `.git` and file `.gitattributes
`
3. Upload the rest of the files in Google Drive - to folder `BgGPT-7B-Instruct-v0.1`


##Mount the folder

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Check for the GPU availability in Colab
Run the following command to verify the GPU is available after changing the runtime to GPU.

In [2]:
!nvidia-smi

Sat Feb 24 23:20:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

##Install Required Python Packages
Google Colab has preinstalled all needed packages except ***accelerate***.

In [3]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.1 MB/s eta 0:00:00


### Load the model and the tokenizer

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/BgGPT-7B-Instruct-v0.1")
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="/content/drive/MyDrive/BgGPT-7B-Instruct-v0.1",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True
)
device = "cuda"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## The prompt and the generation
First time you should wait loading of the model to the GPU

In [26]:
prompt = "Кога и от кого е създадена България" # @param {type: "string"}
prompt = "[s][INST]"+prompt+"[/INST]" # better Question Answering

model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
model.to(device)

generated_ids = model.generate(**model_inputs, temperature=0.1, top_k=1, top_p=1.0, repetition_penalty=1.4, min_new_tokens=100, max_new_tokens=10000, do_sample=True, eos_token_id=2, pad_token_id=2)

tokenizer.batch_decode(generated_ids)[0]


'<s> [s][INST]Кога и от кого е създадена България[/INST]Създаването на съвременната българска държава може да се проследи до 681 г. сл. Хр., когато хан Аспарух обединява славянските племена, живеещи в Мизия (днешна Североизточна България), със своите воини прабългари във федерация известна като Първата Българска империя или Стара Велика България . Това събитие бележи началото на първата документирана поява на народ, идентифициран исторически като „български“.\n\nПрабългарите са били номадски тюркско-говорящ народ, произхождащ някъде от Централна Азия около Монголия. Те мигрирали към Европа през земите северно от Черно море между края на първото хилядолетие пр. н. е. и средата на седми век след това. Първоначално те създават своя собствена малка конфедерация под името Оногондурска орда при кан Кубират близо до река Волга. След смъртта му обаче синовете му разделят тази група на няколко фракции - една част остава там, а друга тръгва заедно с още две групи за Балканите. Именно тези три ча

## Next steps
1. Play with [generate method parameters](https://huggingface.co/docs/transformers/en/main_classes/text_generation)
2. To Do ...

## Sources
1. https://huggingface.co/INSAIT-Institute/BgGPT-7B-Instruct-v0.1
2. https://iamshobhitagarwal.medium.com/running-mistral-7b-large-language-model-on-google-colab-gpu-completely-free-of-cost-fc0a57279de9
3. https://huggingface.co/mistralai/Mistral-7B-v0.1/discussions/13
